In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
from dense import Dense
from convolutional import Convolutional
from reshape import Reshape
from activations import Softmax, Sigmoid
from losses import binary_cross_entropy, binary_cross_entropy_prime, categorical_cross_entropy,  categorical_cross_entropy_prime
from networks import train, predict
from activation import Activation

In [ ]:
class LeakyReLU(Activation):
    def __init__(self, alpha):
        def leaky_relu(x):
            return np.where(x > 0, x, alpha * x)

        def leaky_relu_prime(x):
            return np.where(x > 0, 1, alpha)

        super().__init__(leaky_relu, leaky_relu_prime)


In [ ]:
def preprocess_data(x, y, limit=None):
    # Shuffle dataset
    indices = np.arange(len(x))
    np.random.shuffle(indices)

    if limit:
        indices = indices[:limit]

    x, y = x[indices], y[indices]

    # Reshape to match (channels, height, width)
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255

    # One-hot encode labels (0-9 → 10 classes)
    y = to_categorical(y, num_classes=10)
    y = y.reshape(len(y), 10, 1)

    return x, y

In [ ]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Example: take all data
x_train, y_train = preprocess_data(x_train, y_train, 15000)
x_test, y_test = preprocess_data(x_test, y_test, 800)


In [ ]:
x_train[0]

array([[[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 3),
    LeakyReLU(0.01),
    Convolutional((3, 26, 26), 3, 3),
    LeakyReLU(0.01),
    Reshape((3, 24, 24), (3 * 24 * 24, 1)),
    Dense(3 * 24 * 24, 100),
    LeakyReLU(0.01),
    Dense(100, 10), # Changed output size to 10 to match the number of classes
    Softmax()
    # dense ip 50 reduced accuracy #retry with dense 100 but two conv layers
]

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 3),
    Sigmoid(),
    Convolutional((3, 26, 26), 3, 3),
    Sigmoid(),
    Reshape((3, 24, 24), (3 * 24 * 24, 1)),
    Dense(3 * 24 * 24, 100),
    Sigmoid(),
    Dense(100, 100),
    Sigmoid(),
    Dense(100, 10), # Changed output size to 10 to match the number of classes
    Softmax()
    # dense ip 50 reduced accuracy #retry with dense 100 but two conv layers
]
#best one so far wrt error but laned on 94 ish percent accuracy

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 3),
    Sigmoid(),
    Convolutional((3, 26, 26), 3, 3),
    Sigmoid(),
    Reshape((3, 24, 24), (3 * 24 * 24, 1)),
    Dense(3 * 24 * 24, 100),
    Sigmoid(),
    Dense(100, 10), # Changed output size to 10 to match the number of classes
    Softmax()
    # dense ip 50 reduced accuracy #retry with dense 100 but two conv layers
]
#95.62 latest

In [ ]:
network = [
    Convolutional((1, 28, 28), 3, 3),
    Sigmoid(),
    Reshape((3, 26, 26), (3 * 26 * 26, 1)),
    Dense(3 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 10), # Changed output size to 10 to match the number of classes
    Softmax()
]
# 95

8 layers 2000 data set, 90+ accuracy

7 layers 4000 data set, 90.62 accuracy



In [ ]:
# train
train(
    network,
    categorical_cross_entropy,
    categorical_cross_entropy_prime,
    x_train,
    y_train,
    epochs=50,
    learning_rate=0.01
)

1/50, error=0.03525826981080664
2/50, error=0.012389614394047581
3/50, error=0.007102000390671742
4/50, error=0.005086337878900215
5/50, error=0.004101361164373569
6/50, error=0.0031197296309516814
7/50, error=0.002842463476631632
8/50, error=0.002351970883126922
9/50, error=0.001870116885465538
10/50, error=0.00260223899260868
11/50, error=0.0017335708643255248
12/50, error=0.001620031982334359
13/50, error=0.0013551419940261082
14/50, error=0.0012665619855197385
15/50, error=0.0016389022761904598
16/50, error=0.0015336711037185818
17/50, error=0.002646492665995849
18/50, error=0.0013899170343955433
19/50, error=0.0012911460716370745
20/50, error=0.0013569576284565817
21/50, error=0.0029313829665525855
22/50, error=0.0008880198482115007
23/50, error=0.0005372149721691418
24/50, error=0.0016060330152036894
25/50, error=0.0011687383106212068
26/50, error=0.0007540576224342432
27/50, error=0.0011151596832846378
28/50, error=0.0012858882620413261
29/50, error=0.0012464570501406432
30/50, 

In [ ]:
# test
for x, y in zip(x_test, y_test):
    output = predict(network, x)
    print(f"pred: {np.argmax(output)}, true: {np.argmax(y)}")

pred: 0, true: 0
pred: 3, true: 3
pred: 0, true: 0
pred: 1, true: 1
pred: 9, true: 7
pred: 0, true: 0
pred: 5, true: 5
pred: 7, true: 7
pred: 2, true: 2
pred: 4, true: 4
pred: 3, true: 3
pred: 2, true: 2
pred: 3, true: 3
pred: 4, true: 4
pred: 1, true: 8
pred: 3, true: 3
pred: 0, true: 0
pred: 6, true: 6
pred: 0, true: 0
pred: 6, true: 6
pred: 3, true: 3
pred: 6, true: 6
pred: 2, true: 2
pred: 9, true: 9
pred: 8, true: 8
pred: 6, true: 6
pred: 9, true: 9
pred: 3, true: 3
pred: 7, true: 7
pred: 3, true: 3
pred: 0, true: 0
pred: 7, true: 7
pred: 9, true: 9
pred: 7, true: 7
pred: 9, true: 9
pred: 9, true: 9
pred: 3, true: 3
pred: 1, true: 1
pred: 9, true: 9
pred: 2, true: 2
pred: 2, true: 2
pred: 1, true: 1
pred: 5, true: 5
pred: 1, true: 1
pred: 3, true: 3
pred: 0, true: 0
pred: 6, true: 2
pred: 5, true: 5
pred: 9, true: 9
pred: 8, true: 8
pred: 0, true: 0
pred: 8, true: 8
pred: 4, true: 4
pred: 9, true: 9
pred: 7, true: 7
pred: 0, true: 0
pred: 6, true: 6
pred: 6, true: 6
pred: 4, true:

In [ ]:
correct = 0
total = len(x_test)

for x, y in zip(x_test, y_test):
    output = predict(network, x)
    if np.argmax(output) == np.argmax(y):
        correct += 1

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 96.88%


In [ ]:
def get_all_params(network):
    params = []
    for i, layer in enumerate(network):
        layer_info = {"layer_index": i, "type": type(layer).__name__}
        if hasattr(layer, "kernels"):
            layer_info["kernels"] = layer.kernels
            layer_info["biases"] = layer.biases
        elif hasattr(layer, "weights"):
            layer_info["weights"] = layer.weights
            layer_info["bias"] = layer.bias
        params.append(layer_info)
    return params
